# Capstone project for Victor Herrera

![Course Hero](images/hero.png)

## Introduction

This project is the capstone project for the 'Plating with Jupyter notebooks and Machine learning' bootcamp. The purpose of this project is to improve my data cleaning skills and create a model that is capable of estimating a used car value according to several factors.

## Data Set Selection

The dataset was found on Kaggle. 

It resulted interesting for me because I like to check every now and then the prices of different car models just to have an idea of the current state of the car market. I know that this prices may not be tha same for my country but it's a good place to start.

## Data Examination

Let's start with the imports for the notebook.

Note: Remember to add in the `requirements.txt` file all the modules you use.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Get the selected data set into a pandas Dataframe.

Note: You need to add the right method to load the data.

In [ ]:
cars_raw_df = pd.read_csv('data/cars_raw.csv')

Find relevant information about the selected dataset.

- How many rows and columns does it has?
- Which characteristics does each column has?
    - Data type
    - Minimum and maximum values
    - Values distribution
    - Missing data
- Which columns are related or are dependent on each other? 
    - Which ones can be derived?
    - Which are good candidates for an hypothesis?

Note: Use pandas methods as shape, head, sample, groupby, describe and any other you can think of!

In [ ]:
# How many rows and columns does it has?

print("Data frame has:")
print(" -- " + str(cars_raw_df.shape[1]) + " rows")
print(" -- " + str(cars_raw_df.shape[0]) +" cols ")

print("Columns in the data frame are:")
for col in cars_raw_df.columns:
    print(" -- " + col)

In [ ]:
# Display some data to get an idea of what we are working with

print("5 entries sample:")
cars_raw_df.sample(n=5)

In [ ]:
# Colums data type and missing data

print("Columns composition is:")
cars_raw_df.info(verbose=True)

In [ ]:
# Of which we only have these unique values

cars_raw_df.nunique()

In [ ]:
# Columns data min and max values as well as dristribution
cars_raw_df.describe(include="all")

## Define the Hypothesis to test

The price of an used car should be able to be estimated based on several attributes of the state of the car. Older cars should be cheaper than newer cars, but also a car from a premium brand should have a higher value than a regular brand car from the same year.

The hypothesis to test is that depending on the attributes of the car we should be able to estimate a "fair price" for selling.

### Drawing some charts



## Clean the data

Create a new Data Frame just with the data you are going to use

### Removing columns that i'm not interested in:

- **ConsumerRating** as I'm not sure what the user is rating
- **ConsumerReviews** more reviews shouldn't imply a higher price
- **SellerName** because `cars_raw_df["SellerName"].nunique()` returns 3971, so it would be harder to transform those unique values to a number representation
- **SellerRating** because I'm analizing the cars, not the seller
- **SellerReviews** same as seller rating
- **StreetName** because I'm not interested in the particular seller address
- **Zipcode** because I will use  `State` as my reference for location
- **DealType** because I don't know what is the criteria for a 'Great' deal
- **ValueForMoneyRating** beacause this may be the most subjective value of them all, how would a potential seller rate this value?
- **MinMPG** Related to engine type
- **MaxMPG** Related to engine type
- **VIN** Is unique to each vehicle
- **Stock#** is unique to each vehicle


In [ ]:
# SellerName overview
print("Registered unique seller names:")
cars_raw_df["SellerName"].nunique()

In [ ]:
# DealType overview
print("Types of DealType:")
cars_raw_df["DealType"].unique()

In [ ]:
# Column removal

cars_df = cars_raw_df.drop(columns=[
    "ConsumerRating",
    "ConsumerReviews",
    "SellerName",
    "SellerRating",
    "SellerReviews",
    "StreetName",
    "Zipcode",
    "DealType",
    "ValueForMoneyRating",
    "MinMPG",
    "MaxMPG",
    "VIN",
    "Stock#",
])

cars_df.describe(include="all")

In [ ]:
print("Remaining columns in the data frame are:")
for col in cars_df.columns:
    print(" -- " + col)

### Narrow the data frame

For this excercise we will focus on a certain car model.

First I wan't to know what car manufacturer has the most data.

At this moment my guss would be Toyota because fun fact: the Toyota Corolla is the most sold car ever in history

In [ ]:
#Define a function for listing top n for certain column
def list_top_n(df, col, n):
    return (df.groupby(col)[col]
        .count()
        .reset_index(name='count')
        .sort_values(['count'], ascending=False)
        .head(n))

In [ ]:
print("Top 10 Makers count:")

list_top_n(cars_df, col="Make", n=10)

I guess I was wrong, and the used car market is flodded with BMW's then. I would like to have a look at the top 5 makers

In [ ]:
top_3_makers = ["BMW", "Mercedes-Benz", "Toyota"]
top_5_makers = ["BMW", "Mercedes-Benz", "Toyota", "Honda", "Ford"]
top_makers = cars_df[cars_df["Make"].isin(top_5_makers)]
top_makers.describe(include="all")

In my first attempt I tried to use the top 3 makers and for those (`BMW`, `Mercedes-Benz`, `Toyota`) the most pupular car model was the **BMW X5 xDrive40i**, but I had my doubts. Maybe rich people change care more often, so they resell their car when buying a new one...

![People's car](images/bmw_x5.jpg)

Including the top 5 makers resulted in the most popular car model being the **Honda CR-V EX-L** wich make a little bit more sense to me.

![Honda CR-V](images/honda-cr-v.png)

But the issue persisted, the most common car model for the whole data frame is the **Jeep Grand Cherokee Limited**, so not a Honda nor a BMW

### Model cleanup

Often times a cars model name is the sum of the model itself combined with some base extra commodities that the manufacturer adds on top that base model.

In [ ]:
bmw_df = cars_df[cars_df["Make"] == "BMW"]

list_top_n(bmw_df, col="Model", n=10)

Returning to BMW, we can see that out of the top 10 most popular BMW models 2 are different configurations of the X5 ("X5 xDrive40i", "X5 xDrive35i") and 3 are X3 ("X3 xDrive30i", "X3 sDrive30i", "X3 xDrive28i").

I would like to do the same but for top 30 car models from the whole data frame:

In [ ]:
list_top_n(cars_df, col="Model", n=30)

Reviewing the top 30 models I would make the **bold** assumption that more often than not, the last word is the one that has the most extras, so maybe deleting that last word may produce different results for the same analysis.

In [ ]:
cars_df[["ModelBase", "ModelExtras"]] = cars_df["Model"].str.rsplit(' ', n=1, expand=True)
cars_df.head()

Now that we have our models separated by base and extras let's see what are the top models

So, lets compare the before and after top 10

Top | Model | count | vs | count | ModelBase 
--- | --- | --- | --- | --- | ---
1 | Grand Cherokee Limited | 266| - | 331 | Grand Cherokee
2 | CR-V EX-L | 189| - | 305 | CR-V
3 | X5 xDrive40i | 164| - | 248 | X5
4 | XT5 Luxury | 142| - | 217 | RAV4
5 | RAV4 LE | 112| - | 203 | X3
6 | GX 460 Base | 111| - | 147 | XT5
7 | X3 xDrive30i | 108| - | 127 | RX 350
8 | Pilot EX-L | 102| - | 126 | Pilot
9 | CX-5 Grand Touring | 99| - | 125 | 1500
10 | 1500 Laramie | 88| - | 117 | F-150

In [ ]:
list_top_n(cars_df, col="ModelBase", n=30)

So we will be using the **Jeep Grand Cherokee** for this demo because its base model is the most frequent value on the dataframe.

![Jeep Grand Cherokee](images/jeep_grand_cherokee.png)

In [ ]:
# Select the model and make a copy to work with it inplace

# Work with a certain model
working_df = cars_df[cars_df["ModelBase"] == "Grand Cherokee"].copy()

# Work with all cars on df
#working_df = cars_df.copy()

list_top_n(working_df, col="ModelExtras", n=20)

In [ ]:
working_df.describe(include="all")

In [ ]:
# Drop model
working_df.drop(columns=["Model"], inplace=True)

In [ ]:
## Review Used/New
working_df["Used/New"].unique()

In [ ]:
## Set as certified
def replace_certified(condition):
    if "certified" in condition.lower():
        return "Certified"
    else:
        return "Used"
working_df["Used/New"] = working_df["Used/New"].apply(replace_certified)

In [ ]:
# Change price format
def price_as_number(price):
    if "$" not in price:
        return None
    return float(price.replace("$", "").replace(",", ""))

working_df["Price"] = working_df["Price"].apply(price_as_number)

In [ ]:
# View price distribution
import plotly.express as px

fig = px.histogram(working_df, x="Price")
fig.show()

In [ ]:
# Remove potential outliers
working_df = working_df.loc[working_df["Price"] >= 12000]
working_df = working_df.loc[working_df["Price"] < 56000]

In [ ]:
# Review color
working_df["ExteriorColor"].unique()

In [ ]:
# Review drivetrain values
working_df["Drivetrain"].unique()

In [ ]:
# Replace drivetrain
drivetrains = {
        "Front-wheel Drive": "FWD",
        "Front Wheel Drive": "FWD",
        "Four-wheel Drive": "4WD",
        "Rear-wheel Drive": "RWD",
        "All-wheel Drive": "AWD",
        "4WD": "4WD",
        "AWD": "AWD",
        "RWD": "RWD",
        "FWD": "FWD",
        "–": None,
        None: None
    }
def normalize_drivetrain(value):
    return drivetrains[value]

working_df["Drivetrain"] = working_df["Drivetrain"].apply(normalize_drivetrain)

In [ ]:
# Review FuelType
working_df["FuelType"].unique()


In [ ]:
# Review Transmission
working_df["Transmission"].unique()

In [ ]:
def normalize_transmission(value=""):
    value_lower = value.lower()
    if "cvt" in value_lower or "variable" in value_lower:
        return "CVT"
    if "auto" in value_lower or "a/t" in value_lower or "double" in value_lower or "dual" in value_lower:
        return "Automatic"
    if "manual" in value_lower or "m/t" in value_lower:
        return "Manual"
    return value

working_df["Transmission"] = working_df["Transmission"].apply(normalize_transmission)

In [ ]:
# Review Engine
working_df["Engine"].unique()

In [ ]:
# Simplify engine to simpler categories
## I Do not like this approach as it oversimplifies the engine
def normalize_engine(value=""):
    value_lower = value.lower()
    if "diesel" in value_lower:
        if "turbo" in value_lower:
            return "turbo-diesel"
        else:
            return "diesel"
    if "hybrid" in value_lower or "plug" in value_lower:
        if "turbo" in value_lower:
            return "turbo-hybrid"
        else:
            return "hybrid"
    if "electric" in value_lower:
        return "electric"
    if "turbo" in value_lower:
        return "turbo-gasoline"
    return "gasoline"

working_df["Engine"] = working_df["Engine"].apply(normalize_engine)
working_df["Engine"].unique()

In [ ]:
# Resulting df

working_df = working_df.dropna()
working_df.describe(include="all")

In [ ]:
working_df.describe()

In [ ]:
working_df.info()

## Run your experiment(s)

Describe what your experiment is done, and execute it.

Note: Be generous with your plots!

In [ ]:
# We will use all the columns to predict the Price for that model

# Import the required libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression


In [ ]:
# Use the data without the column we want to predict
X = working_df[["Year", "Make", "ModelBase", "Drivetrain", "FuelType", "Mileage"]]
# Get the column we want to predict
y = working_df["Price"]

In [ ]:
#Transform the data using a one hot encoder fot texts and min max scaler for scaling numbers
column_transformer = make_column_transformer(
    (OneHotEncoder(), ["Make", "ModelBase", "Drivetrain", "FuelType"]),
    (MinMaxScaler(), ["Year", "Mileage"]),
    remainder="passthrough",
    sparse_threshold=0)
# Apply the transformer
X = column_transformer.fit_transform(X)

In [ ]:
X

In [ ]:

# Split the data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
X_train

In [ ]:
type(X_train)


In [ ]:
# Create the training data frame
X_train = pd.DataFrame(data=X_train, columns=column_transformer.get_feature_names_out())
X_train.sample(5)

In [ ]:
# Create the test data frame
X_test = pd.DataFrame(data=X_test, columns=column_transformer.get_feature_names_out())
X_test.sample(5)

In [ ]:
# Define and fit the model
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

In [ ]:
print("Model coeficient and intercept:")
print(model.coef_, model.intercept_)

In [ ]:
# Make some predictions for out test sample
yfit = model.predict(X_test)
print(yfit)

In [ ]:
y_test

In [ ]:
print(format(accuracy_score(y_test, yfit),".4%"))

### Making a plot

In [ ]:
comparison = y_test.to_frame()
comparison["Predicted"] = yfit.tolist()
comparison.describe()

In [ ]:
# Sort prices ascending
(comparison.sort_values(by=["Price"]).plot(
    kind="line",
    x="Price",
    y="Predicted"
))

## Reach a conclusion

**What was the result of your experiment?**

The most accuracy I got on my model was 0.05% which is quite depressing. But reviewing the predictions it appears that the predictions are not that far off the actual car price

**How can it be improved?**

- Use a different way to get the base model
  - Maybe analyze the common pattern and split changing part
- See what properties affect the price the most
- Transmission could be analyzed by
  - Type: Manual, Automatic, CVT
  - No. of gears (Optional)
  - Proprietary technology
- Engine could be simplified and classified
    - Electric
    - Gas
    - Diesel
    - Turbo charged
    - Other extras
- Use a different scale for mileage and year
  - Those numbers may affect more the price than ratings

**Elaborate in one thing you learn during the capstone project.**

Data analysis is fun and reviewing the data set and continue to learn from it is wonderful.

The way to evaluate accuracy in this case should be a little different, I would thing take the prediction more as reference value rather than an actual determinating value.

I believe that there should be a better model to predict the price.

## Congratulations

You have finished the bootcamp!

![Congratulations](images/congratulations.jpg)